# Load bulk data in Python

This example is available in the Jupyter notebook at: [github.com/intrahealth/client-registry-docs/docs/notebooks/load_bulk_data_in_python.ipynb](https://github.com/intrahealth/client-registry-docs/blob/master/docs/notebooks/simple_query.ipynb)

In [1]:
#!/usr/bin/env python3
from pathlib import Path
from requests_pkcs12 import get, post
import pandas as pd
import numpy as np

import recordlinkage

import fhirclient.models.patient as p
import fhirclient.models.humanname as hn
import fhirclient.models.contactpoint as cp
import fhirclient.models.fhirdate as fd
import fhirclient.models.identifier as ident
from fhirclient import client

import json
import time
import itertools

# suppress warning: "Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.SubjectAltNameWarning)

In [2]:
# versions
print("Pandas version: {0}".format(pd.__version__),'\n')
print("Python Record Linkage version: {0}".format(recordlinkage._version.get_versions()['version']),'\n')
print("Numpy version: {0}".format(np.__version__),'\n')
print("FHIR client version: {0}".format(client.__version__),'\n')

Pandas version: 1.0.3 

Python Record Linkage version: 0.14 

Numpy version: 1.18.4 

FHIR client version: 3.2.0 



In [3]:
# path to your git clone of github.com/intrahealth/client-registry
crhome = Path.home() / 'src' / 'github.com' / 'intrahealth' / 'client-registry'
clientcert = crhome / 'server' / 'sampleclientcertificates' / 'openmrs.p12'
servercert = crhome / 'server' / 'certificates' / 'server_cert.pem'
csv_file = crhome / 'tests' / 'uganda_data_v21_20201501.csv'

In [4]:
df_a = pd.read_csv(csv_file)
# df_a = df_a.set_index('rec_id')
print('Number of records :', len(df_a))
print(df_a.head())

Number of records : 5000
         rec_id sex date_of_birth given_name       surname phone_number  \
0  rec-2762-org   f      19671207     zuwena         acile   712 300633   
1  rec-2009-org   f      19761028     zuwena        lusike   772 614594   
2  rec-3269-org   f      19811002     zuwena      mungugeo   772 162632   
3  rec-1609-org   f      19270719    zuraika   akantambira   772 837692   
4  rec-2802-org   m                   zulfas      nyanchwo   782 855101   

       uganda_nin   art_number  
0  CF21927470OWMT   KMC-819708  
1  CF68167355NUZY   KUB-176148  
2  CF50136842UQFQ   MBA-746695  
3  CF68008770HZML   KMC-270901  
4  CM25736526XWGC   KSG-830566  


In [5]:
# some cleaning
df_a['rec_id'] = df_a['rec_id'].str.strip()
df_a['sex'] = df_a['sex'].str.strip()
df_a['given_name'] = df_a['given_name'].str.strip()
df_a['surname'] = df_a['surname'].str.strip()
df_a['date_of_birth'] = df_a['date_of_birth'].str.strip()
df_a['phone_number'] = df_a['phone_number'].str.strip()
df_a['uganda_nin'] = df_a['uganda_nin'].str.strip()
df_a['art_number'] = df_a['art_number'].str.strip()

df_a['sex']= df_a['sex'].replace('f', 'female')
df_a['sex']= df_a['sex'].replace('m', 'male')
print(df_a['sex'].value_counts())

# fhirclient validates and some birthdate fields are empty/improperly formatted
# remove non-digits
df_a['date_of_birth'] = df_a['date_of_birth'].str.extract('(\d+)', expand=False)
# force into datetime (coerce has benefit that it removes anything outside of 8 digits)
df_a['date_of_birth'] =  pd.to_datetime(df_a['date_of_birth'], errors='coerce')
# now back into str or fhirdate will complain
df_a['date_of_birth'] = df_a['date_of_birth'].apply(lambda x: x.strftime('%Y-%m-%d')if not pd.isnull(x) else '')

print(df_a.head())

female    3224
           963
male       809
d            1
r            1
q            1
k            1
Name: sex, dtype: int64
         rec_id     sex date_of_birth given_name      surname phone_number  \
0  rec-2762-org  female    1967-12-07     zuwena        acile   712 300633   
1  rec-2009-org  female    1976-10-28     zuwena       lusike   772 614594   
2  rec-3269-org  female    1981-10-02     zuwena     mungugeo   772 162632   
3  rec-1609-org  female    1927-07-19    zuraika  akantambira   772 837692   
4  rec-2802-org    male                   zulfas     nyanchwo   782 855101   

       uganda_nin  art_number  
0  CF21927470OWMT  KMC-819708  
1  CF68167355NUZY  KUB-176148  
2  CF50136842UQFQ  MBA-746695  
3  CF68008770HZML  KMC-270901  
4  CM25736526XWGC  KSG-830566  


In [6]:
# default server/path
server = "https://localhost:3000/Patient"
# 3 records, modify if more are required
limit = 100
for index, row in itertools.islice(df_a.iterrows(), limit):
# for index, row in df_a.iterrows():
    patient = p.Patient() # not using rec_id as pandas id, leaving empty
    patient.gender = row['sex']
    name = hn.HumanName()
    name.given = [row['given_name']]
    name.family = row['surname']
    name.use = 'official'
    patient.name = [name]
    phone = cp.ContactPoint()
    phone.system = 'phone'
    phone.value = row['phone_number']
    patient.telecom = [phone]
    patient.birthDate = fd.FHIRDate(row['date_of_birth'])
    emr = ident.Identifier()
    emr.system = 'http://clientregistry.org/openmrs'
    emr.value = row['rec_id']
    art = ident.Identifier()
    art.system = 'http://system1/artnumber'
    art.value = row['art_number']
    nin = ident.Identifier()
    nin.system = 'http://system1/nationalid'
    nin.value = row['uganda_nin']
    patient.identifier = [emr, art, nin]
    # print(json.dumps(patient.as_json()))

    headers = {'Content-Type': 'application/json'}
    start = time.time()
    response = post(server, headers=headers, data=json.dumps(patient.as_json()), 
                    pkcs12_filename=clientcert, pkcs12_password='', verify=servercert)
    end = time.time()
    print(index, response.headers['location'], " | ", round((end - start), 1), "ms") # response.headers['Date']
    # print(response.headers)


0 Patient/998bd085-8cac-4d80-a2dc-35dafad28140  |  2.1 ms
1 Patient/7e5bfabb-c77d-403d-b583-14f9c04bcc4f  |  1.6 ms
2 Patient/72a4ad32-4955-4ffc-9b6f-2ba849abcbcf  |  1.8 ms


Failed to initialize FHIRDate from "": Unrecognised ISO 8601 date format: ''


3 Patient/f372452d-d4e5-494d-b87b-da546e2908d7  |  1.3 ms
4 Patient/7c743b88-04f0-4638-b12c-cb45a7604234  |  1.4 ms
5 Patient/6299bc99-a6cf-4a37-8f31-08d7c954bcb6  |  1.5 ms


Failed to initialize FHIRDate from "": Unrecognised ISO 8601 date format: ''


6 Patient/b21337a7-72d2-43a7-8c4b-5c6ddc61e853  |  1.2 ms


...

70 Patient/213c6b3d-5d13-4ad9-b6b4-ab2ceaee324a  |  0.8 ms
71 Patient/3576d266-c676-43ea-9023-bb11c11b2b6b  |  0.9 ms
72 Patient/59369ae7-a50f-4c88-8fa0-7cd30e317d7e  |  0.9 ms
73 Patient/d78ca73d-e8ff-4bd6-a505-ef929161fda6  |  0.7 ms
74 Patient/70277660-d1ec-4fe2-9853-50ad88cc5f7a  |  0.8 ms


Failed to initialize FHIRDate from "": Unrecognised ISO 8601 date format: ''


75 Patient/59369ae7-a50f-4c88-8fa0-7cd30e317d7e  |  0.7 ms
76 Patient/70277660-d1ec-4fe2-9853-50ad88cc5f7a  |  1.2 ms
77 Patient/1261bb98-582e-45cc-8a19-ec172c6c6f4f  |  1.3 ms
78 Patient/9c912e01-5e34-471a-b6c1-b650cc6c2601  |  1.2 ms
79 Patient/d6bc698f-6af2-4578-ad12-ba0bd9d14ed4  |  0.8 ms
80 Patient/1261bb98-582e-45cc-8a19-ec172c6c6f4f  |  1.2 ms
81 Patient/e5426498-f107-447c-ab01-b0b4008732aa  |  0.9 ms
82 Patient/3f6ddd3e-f4fd-45f6-9c60-c16904d9f0aa  |  0.7 ms
83 Patient/a933a54d-e34e-485e-9c95-6cb54f4f2533  |  1.5 ms
84 Patient/609660b5-c1aa-409f-bd9a-cc3d7dac6384  |  1.2 ms
85 Patient/c622e666-fdf6-4d61-a853-3e33d49e2d11  |  0.9 ms


Failed to initialize FHIRDate from "": Unrecognised ISO 8601 date format: ''


86 Patient/86b01ded-7acd-49d8-90e0-63e25662188a  |  1.1 ms
87 Patient/6672ce4d-190a-4be0-9afa-11340b34f9c7  |  1.8 ms
88 Patient/50f359c6-cb99-4a7e-a466-ec97b80225fc  |  0.8 ms
89 Patient/2cf7014c-6a88-42d0-951f-f24375e9c249  |  0.9 ms
90 Patient/f60e5b30-921d-4ca9-b5a8-c00574049ae3  |  1.0 ms
91 Patient/b9e04ae8-0ca2-4c50-b122-5e57448e0d38  |  1.1 ms
92 Patient/4bf4445a-8026-432f-a6ca-f9d277ff7d11  |  1.0 ms
93 Patient/d34127e7-cc18-43df-b45b-12679c286d5d  |  0.8 ms
94 Patient/4bf4445a-8026-432f-a6ca-f9d277ff7d11  |  1.2 ms
95 Patient/a2ed2adf-bb3f-40e5-a05d-c99c89eb3ed5  |  0.9 ms
96 Patient/7fbe823f-5a44-4ef8-9728-98e27f00d038  |  0.8 ms
97 Patient/7506a352-2486-452d-be15-25a40838a48e  |  0.7 ms
98 Patient/54309891-fa19-489e-bb0f-e95360bb7ee0  |  0.8 ms
99 Patient/71e25c0b-1eb3-4739-9264-80e2971dc581  |  0.7 ms
